In [1]:
%matplotlib notebook
import cvxpy as cp
import dccp
import torch
import numpy as np
from cvxpylayers.torch import CvxpyLayer
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import svm
from sklearn.metrics import zero_one_loss, confusion_matrix
from scipy.io import arff
import pandas as pd
import time
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
import matplotlib.patches as mpatches
import json
import random
import math
import os, psutil
from datetime import datetime

torch.set_default_dtype(torch.float64)
torch.manual_seed(0)
np.random.seed(0)

XDIM = 11
COST = 1/XDIM
TRAIN_SLOPE = 1
EVAL_SLOPE = 5
X_LOWER_BOUND = -10
X_UPPER_BOUND = 10

# Utils

In [2]:
def split_data(X, Y, percentage):
    num_val = int(len(X)*percentage)
    return X[num_val:], Y[num_val:], X[:num_val], Y[:num_val]

def shuffle(X, Y):
    data = torch.cat((X, Y), 1)
    data = data[torch.randperm(data.size()[0])]
    X = data[:, :2]
    Y = data[:, 2]
    return X, Y

def conf_mat(Y1, Y2):
    num_of_samples = len(Y1)
    mat = confusion_matrix(Y1, Y2, labels=[-1, 1])*100/num_of_samples
    acc = np.trace(mat)
    return mat, acc

def calc_accuracy(Y, Ypred):
    num = len(Y)
    temp = Y - Ypred
    acc = len(temp[temp == 0])*1./num
    return acc

# Dataset

In [4]:
def load_credit_default_data():
    torch.manual_seed(0)
    np.random.seed(0)
    url = 'https://raw.githubusercontent.com/ustunb/actionable-recourse/master/examples/paper/data/credit_processed.csv'
    df = pd.read_csv(url)
    df["NoDefaultNextMonth"].replace({0: -1}, inplace=True)
    df = df.sample(frac=1).reset_index(drop=True)

    df = df.drop(['Married', 'Single', 'Age_lt_25', 'Age_in_25_to_40', 'Age_in_40_to_59', 'Age_geq_60'], axis = 1)

    scaler = StandardScaler()
    df.loc[:, df.columns != "NoDefaultNextMonth"] = scaler.fit_transform(df.drop("NoDefaultNextMonth", axis=1))

    fraud_df = df.loc[df["NoDefaultNextMonth"] == -1]
    non_fraud_df = df.loc[df["NoDefaultNextMonth"] == 1][:6636]

    normal_distributed_df = pd.concat([fraud_df, non_fraud_df])

    # Shuffle dataframe rows
    df = normal_distributed_df.sample(frac=1).reset_index(drop=True)

    Y, X = df.iloc[:, 0].values, df.iloc[:, 1:].values
    return torch.from_numpy(X), torch.from_numpy(Y)

# CCP classes

In [5]:
class CCP:
    def __init__(self, x_dim, funcs):
        self.f_derivative = funcs["f_derivative"]
        self.g = funcs["g"]
        self.c = funcs["c"]
        
        self.x = cp.Variable(x_dim)
        self.xt = cp.Parameter(x_dim)
        self.r = cp.Parameter(x_dim)
        self.w = cp.Parameter(x_dim)
        self.b = cp.Parameter(1)
        self.slope = cp.Parameter(1)

        target = self.x@self.f_derivative(self.xt, self.w, self.b, self.slope)-self.g(self.x, self.w, self.b, self.slope)-self.c(self.x, self.r)
        constraints = [self.x >= X_LOWER_BOUND,
                       self.x <= X_UPPER_BOUND]
        self.prob = cp.Problem(cp.Maximize(target), constraints)
        
    def ccp(self, r):
        """
        numpy to numpy
        """
        self.xt.value = r
        self.r.value = r
        result = self.prob.solve()
        diff = np.linalg.norm(self.xt.value - self.x.value)
        cnt = 0
        while diff > 0.0001 and cnt < 10:
            cnt += 1
            self.xt.value = self.x.value
            result = self.prob.solve()
            diff = np.linalg.norm(self.x.value - self.xt.value)
        return self.x.value
    
    def optimize_X(self, X, w, b, slope):
        """
        tensor to tensor
        """
        w = w.detach().numpy()
        b = b.detach().numpy()
        slope = np.full(1, slope)
        X = X.numpy()
        
        self.w.value = w
        self.b.value = b
        self.slope.value = slope
        
        return torch.stack([torch.from_numpy(self.ccp(x)) for x in X])

In [6]:
class DELTA():
    
    def __init__(self, x_dim, funcs):
        self.g = funcs["g"]
        self.c = funcs["c"]
        
        self.x = cp.Variable(x_dim)
        self.r = cp.Parameter(x_dim, value = np.random.randn(x_dim))
        self.w = cp.Parameter(x_dim, value = np.random.randn(x_dim))
        self.b = cp.Parameter(1, value = np.random.randn(1))
        self.f_der = cp.Parameter(x_dim, value = np.random.randn(x_dim))

        target = self.x@self.f_der-self.g(self.x, self.w, self.b, TRAIN_SLOPE)-self.c(self.x, self.r)
        constraints = [self.x >= X_LOWER_BOUND,
                       self.x <= X_UPPER_BOUND]
        objective = cp.Maximize(target)
        problem = cp.Problem(objective, constraints)
        self.layer = CvxpyLayer(problem, parameters=[self.r, self.w, self.b, self.f_der],
                                variables=[self.x])
        
        
    def optimize_X(self, X, w, b, F_DER):
        return self.layer(X, w, b, F_DER)[0]

# Gain & Cost functions

In [7]:
def score(x, w, b):
    return x@w + b

def f(x, w, b, slope):
    return 0.5*cp.norm(cp.hstack([1, (slope*score(x, w, b) + 1)]), 2)

def g(x, w, b, slope):
    return 0.5*cp.norm(cp.hstack([1, (slope*score(x, w, b) - 1)]), 2)

def c(x, r):
    return COST*cp.sum_squares(x-r)

def f_derivative(x, w, b, slope):
    return 0.5*cp.multiply(slope*((slope*score(x, w, b) + 1)/cp.sqrt((slope*score(x, w, b) + 1)**2 + 1)), w)

funcs = {"f": f, "g": g, "f_derivative": f_derivative, "c": c, "score": score}

# Data generation

In [8]:
X, Y = load_credit_default_data()
X, Y = X[:3000], Y[:3000]
assert(len(X[0]) == XDIM)
X, Y, Xval, Yval = split_data(X, Y, 0.5)
Xval, Yval, Xtest, Ytest = split_data(Xval, Yval, 0.5)

print("percent of positive samples: {}%".format(100 * len(Y[Y == 1]) / len(Y)))
visualize_data(X, Y)

11
percent of positive samples: 49.266666666666666%


# Model

In [9]:
class MyStrategicModel(torch.nn.Module):
    def __init__(self, x_dim, funcs, train_slope, eval_slope, strategic=False, lamb=0):
        torch.manual_seed(0)
        np.random.seed(0)
        super(MyStrategicModel, self).__init__()
        self.x_dim = x_dim
        self.train_slope, self.eval_slope = train_slope, eval_slope
        self.w = torch.nn.parameter.Parameter(math.sqrt(1/x_dim)*(1-2*torch.rand(x_dim, dtype=torch.float64, requires_grad=True)))
        self.b = torch.nn.parameter.Parameter(torch.rand(1, dtype=torch.float64, requires_grad=True))
        self.strategic = strategic
        self.lamb = lamb
        self.ccp = CCP(x_dim, funcs)
        self.delta = DELTA(x_dim, funcs)

    def forward(self, X, evaluation=False):
        if evaluation:
            XT = self.ccp.optimize_X(X, self.w, self.b, self.eval_slope)
            utility = self.calc_utility(X, XT, evaluation=True)
            X_opt = XT
        else:
            XT = self.ccp.optimize_X(X, self.w, self.b, self.train_slope)
            F_DER = self.get_f_ders(XT, self.train_slope)
            X_opt = self.delta.optimize_X(X, self.w, self.b, F_DER) # Xopt should equal to XT but we do it again for the gradients
            utility = self.calc_utility(X, X_opt, evaluation=False)
        
        if self.strategic:
            output = self.score(X_opt)
        else:
            output = self.score(X)        
        
        return output, utility
    
    def optimize_X(self, X):
        return self.ccp.optimize_X(X, self.w, self.b, self.eval_slope)
    
    def score(self, x):
        return x@self.w + self.b
    
    def get_f_ders(self, XT, slope):
        return torch.stack([0.5*slope*((slope*self.score(xt) + 1)/torch.sqrt((slope*self.score(xt) + 1)**2 + 1))*self.w for xt in XT])

    def evaluate(self, X, Y):
        scores, _ = self.forward(X, evaluation=True)
        Y_pred = torch.sign(scores)
        num = len(Y)
        temp = Y - Y_pred
        acc = len(temp[temp == 0])*1./num        
        return acc
    
    def calc_utility(self, X, X_opt, evaluation=False):
        return torch.mean(self.calc_utility_aux(X, X_opt, evaluation))
    
    def calc_utility_aux(self, X, X_opt, evaluation=False):
        """
        NOT GENERIC FOR f, g, c!!!
        """
        slope = self.eval_slope if evaluation else self.train_slope
        S = self.score(X_opt)
        gain = 0.5*(torch.sqrt((slope*S + 1)**2 + 1) - torch.sqrt((slope*S - 1)**2 + 1))
        cost = COST*torch.sum((X_opt-X)**2, dim=1)
        
        return gain - cost
    
    def loss(self, Y, Y_pred, utility):
        if self.lamb > 0:
            return torch.mean(torch.clamp(1 - Y_pred * Y, min=0)) - self.lamb*utility
        else:
            return torch.mean(torch.clamp(1 - Y_pred * Y, min=0))
        
    
    def save_model(self, X, Y, Xval, Yval, Xtest, Ytest, train_errors, val_errors, train_losses, val_losses, val_utilities, info, path, comment=None):
        if comment is not None:
            path += "_____" + comment
            
        filename = path + "/model.pt"
        if not os.path.exists(os.path.dirname(filename)):
            os.makedirs(os.path.dirname(filename))
        torch.save(self.state_dict(), filename)
        
        pd.DataFrame(X.numpy()).to_csv(path + '/X.csv')
        pd.DataFrame(Y.numpy()).to_csv(path + '/Y.csv')
        pd.DataFrame(Xval.numpy()).to_csv(path + '/Xval.csv')
        pd.DataFrame(Yval.numpy()).to_csv(path + '/Yval.csv')
        pd.DataFrame(Xval.numpy()).to_csv(path + '/Xtest.csv')
        pd.DataFrame(Yval.numpy()).to_csv(path + '/Ytest.csv')
        
        pd.DataFrame(np.array(train_errors)).to_csv(path + '/train_errors.csv')
        pd.DataFrame(np.array(val_errors)).to_csv(path + '/val_errors.csv')
        pd.DataFrame(np.array(train_losses)).to_csv(path + '/train_losses.csv')
        pd.DataFrame(np.array(val_losses)).to_csv(path + '/val_losses.csv')
        pd.DataFrame(np.array(val_utilities)).to_csv(path + '/val_utilities.csv')
        
        with open(path + "/info.txt", "w") as f:
            f.write(info)
    
    def load_model(self, filename):
        self.load_state_dict(torch.load(filename))
        self.eval()
    
    def fit(self, X, Y, Xval, Yval, Xtest, Ytest, opt, opt_kwargs={"lr":1e-3}, batch_size=128, epochs=100, verbose=False, callback=None, calc_train_errors=False, comment=None):
        train_dset = TensorDataset(X, Y)
        train_loader = DataLoader(train_dset, batch_size=batch_size, shuffle=True)
        opt = opt(self.parameters(), **opt_kwargs)

        train_losses = []
        val_losses = []
        train_errors = []
        val_errors = []
        val_utilities = []
        
        best_val_error = 1
        consecutive_no_improvement = 0
        now = datetime.now()
        path = "./models/utility/" + now.strftime("%d-%m-%Y_%H-%M-%S")

        total_time = time.time()
        for epoch in range(epochs):
            t1 = time.time()
            batch = 1
            train_losses.append([])
            train_errors.append([])
            for Xbatch, Ybatch in train_loader:
                opt.zero_grad()
                Ybatch_pred, utility = self.forward(Xbatch)
                l = self.loss(Ybatch, Ybatch_pred, utility)
                l.backward()
                opt.step()
                train_losses[-1].append(l.item())
                if calc_train_errors:
                    with torch.no_grad():
                        e = self.evaluate(Xbatch, Ybatch)
                        train_errors[-1].append(1-e)
                if verbose:
                    print("batch %03d / %03d | loss: %3.5f | utility: %3.5f" %
                          (batch, len(train_loader), np.mean(train_losses[-1]), utility))
                batch += 1
                if callback is not None:
                    callback()

            with torch.no_grad():
                Yval_pred, val_utility = self.forward(Xval , evaluation=True)
                val_utility = val_utility.item()
                val_loss = self.loss(Yval, Yval_pred, val_utility).item()
                val_losses.append(val_loss)
                val_error = 1-self.evaluate(Xval, Yval)
                val_errors.append(val_error)
                val_utilities.append(val_utility)
                if val_error < best_val_error:
                    consecutive_no_improvement = 0
                    best_val_error = val_error
                    if self.strategic:
                        info = "training time in seconds: {}\nepoch: {}\nbatch size: {}\ntrain slope: {}\neval slope: {}\nlearning rate: {}\nvalidation loss: {}\nvalidation error: {}\nutility: {}".format(
                        time.time()-total_time, epoch, batch_size, self.train_slope, self.eval_slope, opt_kwargs["lr"], val_loss, val_error, val_utility)
                        self.save_model(X, Y, Xval, Yval, Xtest, Ytest, train_errors, val_errors, train_losses, val_losses, val_utilities, info, path, comment)
                        print("model saved!")
                else:
                    consecutive_no_improvement += 1
                    if consecutive_no_improvement >= 4:
                        break
                
            t2 = time.time()
            if verbose:
                print("----- epoch %03d / %03d | time: %03d sec | loss: %3.5f | utility: %3.5f | err: %3.5f" % (epoch + 1, epochs, t2-t1, val_losses[-1], val_utilities[-1], val_errors[-1]))
        print("training time: {} seconds".format(time.time()-total_time)) 
        return train_errors, val_errors, train_losses, val_losses

# Train

In [120]:
EPOCHS = 10
BATCH_SIZE = 64

x_dim = XDIM

lambda_range = torch.logspace(start=-0.4, end=-0.2, steps=10)
for lamb in lambda_range:

    # strategic classification
    print("---------- training strategically----------")
    print("lambda: ", lamb.item())
    strategic_model = MyStrategicModel(x_dim, funcs, TRAIN_SLOPE, EVAL_SLOPE, strategic=True, lamb=lamb)

    fit_res_strategic = strategic_model.fit(X, Y, Xval, Yval, Xtest, Ytest,
                                    opt=torch.optim.Adam, opt_kwargs={"lr": 5*(1e-2)},
                                    batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=True, calc_train_errors=False,
                                    comment="utility_" + str(lamb.item())

C:\Users\sagil\Anaconda3\envs\funcPred\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\sagil\Anaconda3\envs\funcPred\lib\site-packages\cvxpy\expressions\expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)
C:\Users\sagil\Anaconda3\envs\funcPred\lib\site-packages\cvxpy\expressions\expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix

tensor([0.5092, 0.5092, 0.5093, 0.5093, 0.5093])
---------- training strategically----------
lambda:  0.50921
batch 001 / 024 | loss: 0.64009 | utility: 0.34401
batch 002 / 024 | loss: 0.71321 | utility: 0.43313
batch 003 / 024 | loss: 0.69744 | utility: 0.47862
batch 004 / 024 | loss: 0.70412 | utility: 0.50905
batch 005 / 024 | loss: 0.74470 | utility: 0.44587
batch 006 / 024 | loss: 0.74547 | utility: 0.47644
batch 007 / 024 | loss: 0.75639 | utility: 0.40600
batch 008 / 024 | loss: 0.74481 | utility: 0.40807
batch 009 / 024 | loss: 0.75285 | utility: 0.37149
batch 010 / 024 | loss: 0.74755 | utility: 0.33809
batch 011 / 024 | loss: 0.73015 | utility: 0.27690
batch 012 / 024 | loss: 0.73392 | utility: 0.28286
batch 013 / 024 | loss: 0.72675 | utility: 0.39800
batch 014 / 024 | loss: 0.72833 | utility: 0.22206
batch 015 / 024 | loss: 0.72511 | utility: 0.25626
batch 016 / 024 | loss: 0.72382 | utility: 0.23553
batch 017 / 024 | loss: 0.72496 | utility: 0.26337
batch 018 / 024 | loss:

C:\Users\sagil\Anaconda3\envs\funcPred\lib\site-packages\cvxpy\problems\problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "


----- epoch 004 / 010 | time: 242 sec | loss: 0.46108 | utility: 0.57862 | err: 0.38267
batch 001 / 024 | loss: 0.78361 | utility: 0.28060
batch 002 / 024 | loss: 0.71137 | utility: 0.32763
batch 003 / 024 | loss: 0.75009 | utility: 0.22151
batch 004 / 024 | loss: 0.66364 | utility: 0.28513
batch 005 / 024 | loss: 0.67046 | utility: 0.23328
batch 006 / 024 | loss: 0.65161 | utility: 0.29597
batch 007 / 024 | loss: 0.65554 | utility: 0.23398
batch 008 / 024 | loss: 0.64275 | utility: 0.31441
batch 009 / 024 | loss: 0.65850 | utility: 0.26605
batch 010 / 024 | loss: 0.64627 | utility: 0.22904
batch 011 / 024 | loss: 0.64555 | utility: 0.29628
batch 012 / 024 | loss: 0.65680 | utility: 0.24759
batch 013 / 024 | loss: 0.64739 | utility: 0.22264
batch 014 / 024 | loss: 0.64475 | utility: 0.24197
batch 015 / 024 | loss: 0.64740 | utility: 0.18418
batch 016 / 024 | loss: 0.65308 | utility: 0.18664
batch 017 / 024 | loss: 0.64667 | utility: 0.31583
batch 018 / 024 | loss: 0.64630 | utility: 0.

C:\Users\sagil\Anaconda3\envs\funcPred\lib\site-packages\cvxpy\expressions\expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)
C:\Users\sagil\Anaconda3\envs\funcPred\lib\site-packages\cvxpy\expressions\expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)
C:\Users\sagil\Anaconda3\envs\funcPred\l

...

C:\Users\sagil\Anaconda3\envs\funcPred\lib\site-packages\cvxpy\expressions\expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)
C:\Users\sagil\Anaconda3\envs\funcPred\lib\site-packages\cvxpy\expressions\expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)
C:\Users\sagil\Anaconda3\envs\funcPred\l

batch 001 / 024 | loss: 0.64006 | utility: 0.34401
batch 002 / 024 | loss: 0.71318 | utility: 0.43313
batch 003 / 024 | loss: 0.69741 | utility: 0.47862
batch 004 / 024 | loss: 0.70409 | utility: 0.50906
batch 005 / 024 | loss: 0.74467 | utility: 0.44589
batch 006 / 024 | loss: 0.74544 | utility: 0.47646
batch 007 / 024 | loss: 0.75636 | utility: 0.40604
batch 008 / 024 | loss: 0.74479 | utility: 0.40812
batch 009 / 024 | loss: 0.75282 | utility: 0.37155
batch 010 / 024 | loss: 0.74753 | utility: 0.33815
batch 011 / 024 | loss: 0.73013 | utility: 0.27697
batch 012 / 024 | loss: 0.73390 | utility: 0.28293
batch 013 / 024 | loss: 0.72674 | utility: 0.39807
batch 014 / 024 | loss: 0.72831 | utility: 0.22215
batch 015 / 024 | loss: 0.72509 | utility: 0.25634
batch 016 / 024 | loss: 0.72380 | utility: 0.23562
batch 017 / 024 | loss: 0.72494 | utility: 0.26348
batch 018 / 024 | loss: 0.73600 | utility: 0.23119
batch 019 / 024 | loss: 0.73774 | utility: 0.27661
batch 020 / 024 | loss: 0.73221

C:\Users\sagil\Anaconda3\envs\funcPred\lib\site-packages\cvxpy\problems\problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "


batch 001 / 024 | loss: 0.60517 | utility: 0.30702
batch 002 / 024 | loss: 0.67261 | utility: 0.24297
batch 003 / 024 | loss: 0.67576 | utility: 0.22441
batch 004 / 024 | loss: 0.66106 | utility: 0.23430
batch 005 / 024 | loss: 0.67238 | utility: 0.37272
batch 006 / 024 | loss: 0.69674 | utility: 0.23355
batch 007 / 024 | loss: 0.68985 | utility: 0.22607
batch 008 / 024 | loss: 0.67373 | utility: 0.28567
batch 009 / 024 | loss: 0.66889 | utility: 0.26218
batch 010 / 024 | loss: 0.66442 | utility: 0.24260
batch 011 / 024 | loss: 0.66896 | utility: 0.22022
batch 012 / 024 | loss: 0.66696 | utility: 0.22121
batch 013 / 024 | loss: 0.66403 | utility: 0.24282
batch 014 / 024 | loss: 0.65913 | utility: 0.27735
batch 015 / 024 | loss: 0.65964 | utility: 0.10402
batch 016 / 024 | loss: 0.65807 | utility: 0.27679
batch 017 / 024 | loss: 0.65900 | utility: 0.26053
batch 018 / 024 | loss: 0.65606 | utility: 0.30669
batch 019 / 024 | loss: 0.65807 | utility: 0.25552
batch 020 / 024 | loss: 0.65623

batch 007 / 024 | loss: 0.65880 | utility: 0.19914
batch 008 / 024 | loss: 0.66105 | utility: 0.04109
batch 009 / 024 | loss: 0.67898 | utility: 0.10038
batch 010 / 024 | loss: 0.66957 | utility: 0.09900
batch 011 / 024 | loss: 0.64752 | utility: 0.18316
batch 012 / 024 | loss: 0.63277 | utility: 0.17276
batch 013 / 024 | loss: 0.62976 | utility: 0.18418
batch 014 / 024 | loss: 0.64198 | utility: 0.17343
batch 015 / 024 | loss: 0.64085 | utility: 0.20951
batch 016 / 024 | loss: 0.63683 | utility: 0.19899
batch 017 / 024 | loss: 0.63025 | utility: 0.18373
batch 018 / 024 | loss: 0.63423 | utility: 0.10747
batch 019 / 024 | loss: 0.63804 | utility: 0.20577
batch 020 / 024 | loss: 0.64097 | utility: 0.08128
batch 021 / 024 | loss: 0.64713 | utility: 0.11411
batch 022 / 024 | loss: 0.64931 | utility: 0.14104
batch 023 / 024 | loss: 0.64768 | utility: 0.19052
batch 024 / 024 | loss: 0.65423 | utility: 0.25853
----- epoch 008 / 010 | time: 278 sec | loss: 0.49218 | utility: 0.43712 | err: 0.